In [1]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd

In [2]:
relief_response = requests.get('https://www.baseball-reference.com/leagues/majors/2021-reliever-pitching.shtml')

In [3]:
relief_soup = BeautifulSoup(relief_response.text)
relief_table = relief_soup.select('#teams_reliever_pitching .right , #teams_reliever_pitching .left, #teams_reliever_pitching .center')

In [4]:
col_names = ['Year']
for col_name in relief_table[0:31]:
    col_names.append(col_name.text)
    
ReliefPitching = pd.DataFrame(columns=col_names)
ReliefPitching

,Year,162,565,159,22,41,50,22,28,44%,...,293,142,423,98,153,110,94,3.2,18,161


In [5]:
postseason_response = requests.get('https://www.baseball-reference.com/postseason/')

In [6]:
postseason_soup = BeautifulSoup(postseason_response.text)
postseason_data = postseason_soup.select('#postseason_series a')

In [7]:
starting_pos = 0 
ending_pos = 3
post_names = ['Year', 'Winner', 'Loser']
PostTeams = pd.DataFrame(columns=post_names)
for playoff_series in range(1, 181):
    post_list = []
    for element in postseason_data[starting_pos:ending_pos]:
        post_list.append(element.text)
    post_Series = pd.Series(post_list, index=post_names)
    PostTeams = PostTeams.append(post_Series, ignore_index=True)
    starting_pos += 3
    ending_pos += 3

PostTeams.Year = PostTeams.Year.str.extract('(\d{4})+')
PostTeams.Year = PostTeams.Year.astype('int')
PostTeams.Winner = PostTeams.Winner.str.extract('([\w\s]+\.?[\w\s]+)\*? \(')
PostTeams.Loser = PostTeams.Loser.str.extract('([\w\s]+\.?[\w\s]+)\*? \(')
PostTeams

,Year,Winner,Loser
0,2021,Atlanta Braves,Houston Astros
1,2021,Houston Astros,Boston Red Sox
2,2021,Atlanta Braves,Los Angeles Dodgers
3,2021,Houston Astros,Chicago White Sox
4,2021,Boston Red Sox,Tampa Bay Rays
...,...,...,...
175,2000,New York Mets,St. Louis Cardinals
176,2000,New York Yankees,Oakland Athletics
177,2000,Seattle Mariners,Chicago White Sox
178,2000,St. Louis Cardinals,Atlanta Braves


In [8]:
for year in range(2000, 2022):
    relief_response = requests.get(f'https://www.baseball-reference.com/leagues/majors/{year}-reliever-pitching.shtml')
    relief_soup = BeautifulSoup(relief_response.text)
    relief_table = relief_soup.select('#teams_reliever_pitching .right , #teams_reliever_pitching .left, #teams_reliever_pitching .center')
    
    starting_pos = 0
    ending_pos = 31
    for team_row in range(1, 31):
        starting_pos += 31
        ending_pos += 31
        relief_row = [year]
        for cell in relief_table[starting_pos:ending_pos]:
            relief_row.append(cell.text)
        row_Series = pd.Series(relief_row, index=col_names)
        ReliefPitching = ReliefPitching.append(row_Series, ignore_index=True)
ReliefPitching

,Year,162,565,159,22,41,50,22,28,44%,...,293,142,423,98,153,110,94,3.2,18,161
0,2000,390,146,23,24,56,38,18,68%,97,...,129,261,120,111,140,72,3.4,19,162,376
1,2000,149,19,18,67,53,14,79%,130,56,...,264,91,101,95,81,3.2,18,162,396,148
2,2000,22,21,57,33,24,58%,128,68,255,...,99,125,119,84,3.3,19,162,425,155,29
3,2000,23,66,46,20,70%,117,49,224,84,...,91,191,80,4.0,21,162,421,152,25,35
4,2000,68,39,29,57%,127,52,262,111,42%,...,140,90,3.4,20,162,466,157,30,21,67
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
655,2021,198,73,4.0,21,162,507,162,27,31,...,116,261,179,59,269,143,364,144,98,162
656,2021,50,3.7,21,162,537,161,30,31,52,...,271,245,84,208,167,370,136,136,167,63
657,2021,3.2,18,162,569,161,23,42,73,36,...,243,78,248,151,418,100,164,113,138,3.0
658,2021,18,162,556,160,36,30,66,40,26,...,84,227,150,407,124,126,140,95,3.3,18


In [9]:
playoff_status = []
for year in range(2000, 2022):
    year_teams = ReliefPitching.loc[ReliefPitching['Year'] == year, 'Tm'].unique()
    year_post_teams = PostTeams.loc[PostTeams['Year'] == year, :]
    unique_year_post_teams = pd.concat([year_post_teams.Winner, year_post_teams.Loser], ignore_index=True).unique()
    for team in year_teams:
        if team in unique_year_post_teams:
            playoff_status.append(1)
        else:
            playoff_status.append(0)
            
ReliefPitching['Playoff'] = pd.Series(playoff_status)
len(playoff_status)

KeyError: ignored

In [ ]:
ReliefPitching

In [ ]:
# ReliefPitching.to_csv('ReliefPitching.csv', index = False)